# Using U-net style architecture for semantic segmentation
I will train a u-net-style architecture on the labeled city dataset from 

### Params

In [0]:
batch_size = 32

In [0]:
# get colab status
try:
  import google.colab
  IN_COLAB = True
  %tensorflow_version 2.x
except:
  IN_COLAB = False

TensorFlow 2.x selected.


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras.layers import (Dense, Conv2D, Flatten, Dropout, 
MaxPooling2D, BatchNormalization, Conv2DTranspose, concatenate, Input)
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import RMSprop
from tensorflow.python.keras.utils.data_utils import Sequence # to fix 'imagedatagenerator has no shape' error
import os, zipfile, glob
from math import ceil 

In [0]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir(r'/content/gdrive/My Drive/thinkful/colab_datasets/sidewalk_data/')
else:
    os.chdir(os.path.expanduser(r'~/Google Drive/thinkful/colab_datasets/sidewalk_data/'))
!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/thinkful/colab_datasets/sidewalk_data


# Data Load
### Cityscape Dataset
This is a publically availible dataset from https://www.cityscapes-dataset.com/, though not downloadable without requesting a login which is granted based on email domain.
Citation:  
>Cvpr2016M. Cordts, M. Omran, S. Ramos, T. Rehfeld, M. Enzweiler, R. Benenson, U. Franke, S. Roth, and B. Schiele, “The Cityscapes Dataset for Semantic Urban Scene Understanding,” in Proc. of the IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2016. [Bibtex]
[main paper](https://www.cityscapes-dataset.com/wordpress/wp-content/papercite-data/pdf/cordts2016cityscapes.pdf) · [supplemental ](https://www.cityscapes-dataset.com/wordpress/wp-content/papercite-data/pdf/cordts2016cityscapes-supplemental.pdf)· [arxiv](http://arxiv.org/abs/1604.01685) · [CVF](http://www.cv-foundation.org/openaccess/content_cvpr_2016/html/Cordts_The_Cityscapes_Dataset_CVPR_2016_paper.html)


In [0]:
if 0: # run once
  os.chdir(r'/content/gdrive/My Drive/thinkful/colab_datasets/sidewalk_data/public_data/')
  with zipfile.ZipFile(os.path.join(os.getcwd(), 'gtFine_trainvaltest.zip'), 'r') as zip_ref:
    zip_ref.extractall("./cityscape/")

In [0]:
data_mount = r'/content/gdrive/My Drive/thinkful/colab_datasets/sidewalk_data/public_data'
if 1:
  

test  train  val


### Data Generator

In [0]:
#switch to cloud dir
if IN_COLAB:
    cityscape_path = r'/content/gdrive/My Drive/thinkful/colab_datasets/sidewalk_data/public_data/cityscape/leftImg8bit/'
    mask_path = r'/content/gdrive/My Drive/thinkful/colab_datasets/sidewalk_data/public_data/cityscape/gtFine/'
    common_path = r'/content/gdrive/My Drive/thinkful/colab_datasets/sidewalk_data/public_data/cityscape/'
else:
    cityscape_path = os.path.expanduser(
      r'~/Google Drive/thinkful/colab_datasets/sidewalk_data/public_data/cityscape/leftImg8bit/')
os.chdir(cityscape_path)

In [0]:
#grab name of every image and store in dataframe with subset eg. train, test
data_df = None
data_df = pd.DataFrame(columns=['img_name', 'subset'])
img_paths = os.listdir(os.path.curdir)
for subsetdir in img_paths:
  img_files = [x[len(subsetdir)+1:] for x in glob.glob(os.path.join(subsetdir, '*/*'))]
  img_files_s = pd.Series(img_files)
  new_df = None
  new_df = pd.DataFrame(columns=['img_name', 'subset'])
  new_df['img_name'] = img_files_s
  new_df['subset'].fillna(subsetdir, inplace=True)
  data_df = data_df.append(new_df)

test


In [0]:
#grab name of mask for each image
mask_df = None
mask_df = pd.DataFrame(columns=['mask_name', 'subset'])
mask_paths = os.listdir(mask_path)
os.chdir(mask_path)
for subsetdir in mask_paths:
  mask_files = [x[len(subsetdir)+1:] for x in glob.glob(os.path.join(subsetdir, '*/*Fine_color.png'))]
  mask_files_s = pd.Series(mask_files)
  new_mask_df = None
  new_mask_df = pd.DataFrame(columns=['mask_name', 'subset'])
  new_mask_df['mask_name'] = mask_files_s
  new_mask_df['subset'].fillna(subsetdir, inplace=True)
  mask_df = mask_df.append(new_mask_df, sort=False)

In [0]:
def clean_name(name):
  '''removes _gtFine_color.png or _leftImg8bit.png'''
  which = name.rfind('_leftImg8bit.png')
  if which == -1:
    which = '_gtFine_color.png'
  else:
    which =  '_leftImg8bit.png'
  loc=name.rfind(which)
  return name[:loc]

In [0]:
data_df['common_name'] = data_df.img_name.apply(clean_name)
mask_df['common_name'] = mask_df['mask_name'].apply(clean_name)

In [0]:
data_full_df = None
data_full_df = pd.merge(left=data_df, right=mask_df, 
                        how='inner', on='common_name',
                        sort=False, 
                        suffixes=('_data', '_mask'))[['img_name',
                                                      'subset_data', 
                                                      'mask_name']]

In [0]:
data_full_df.head()

,img_name,subset_data,mask_name
0,berlin/berlin_000001_000019_leftImg8bit.png,test,berlin/berlin_000001_000019_gtFine_color.png
1,berlin/berlin_000005_000019_leftImg8bit.png,test,berlin/berlin_000005_000019_gtFine_color.png
2,berlin/berlin_000007_000019_leftImg8bit.png,test,berlin/berlin_000007_000019_gtFine_color.png
3,berlin/berlin_000014_000019_leftImg8bit.png,test,berlin/berlin_000014_000019_gtFine_color.png
4,berlin/berlin_000000_000019_leftImg8bit.png,test,berlin/berlin_000000_000019_gtFine_color.png


In [0]:
data_full_df[data_full_df.subset_data == "train"]

,img_name,subset_data,mask_name


In [0]:
class DataGenerator(Sequence):
    """
    parent Sequence object generates data batches
    """
    def __init__(self, data, batch_size, data_path, subset): 
        self.data = data[data.subset_data == "subset"]
        self.data_path = data_path
        self.batch_size = batch_size
        self.image_list = data.img_name.to_list()
        self.mask_list = data.mask_name.to_list()
        self.subset = subset
       
    def __len__(self):
      """last batch usually smaller"""
      return int(np.ceil(len(self.data.shape[0]) / float(self.batch_size)))

    def get_batch_images(self, idx, path_list, img_or_mask):
        # Fetch a batch of images from a list of paths
        i_or_m = img_or_mask
        if i_or_m == "img":
          path_l = os.path.join(self.data_path, "leftImg8bit", self.subset)
        else:
          path_l = os.path.join(self.data.path, "gtFine", self.subset)
        list_of_img = [load_img(os.path.join(path_l, im)) for \
                       im in path_list[idx * self.batch_size: (1 + idx) * self.batch_size]]
        return np.array(list_of_img)
    def __getitem__(self, idx):
        batch_x = self.get_batch_images(idx, self.image_list, 'img')
        batch_y = self.get_batch_labels(idx, self.mask_list, 'mask')
        return batch_x, batch_y

In [0]:
DataGenerator(data_full_df, batch_size=32, data_path=common_path, subset='test')[0]

TypeError: ignored

In [0]:
idg_params={'rescale': 1./255}
target_size = (256, 256)
augmentation_params = {'rotation_range': 15,
                        'zoom_range': 0.15,
                         'width_shift_range': 0.2,
                         'height_shift_range': 0.2,
                         'shear_range': 0.15,
                         'fill_mode': 'nearest',
                         'horizontal_flip': True}
datagen_params = {'batch_size': batch_size,
                  'target_size': target_size,
                  'color_mode': 'rgb', 
                  'class_mode': 'binary'}

In [0]:
train_datagen = ImageDataGenerator(  
    **idg_params)
vdate_datagen = ImageDataGenerator(
    **idg_params)
test_datagen = ImageDataGenerator(
    **idg_params)

In [0]:
os.chdir(r'/content/gdrive/My Drive/thinkful/colab_datasets/sidewalk_data/public_data/cityscape/gtFine')

In [0]:
train_generator = train_datagen.flow_from_directory(
        directory=r'./train/',
        **datagen_params)
vdate_generator = vdate_datagen.flow_from_directory(
        directory=r'./val/',
        **datagen_params)
test_generator = test_datagen.flow_from_directory(
        directory=r'./test/',
        **datagen_params)

Found 8925 images belonging to 18 classes.
Found 1500 images belonging to 3 classes.
Found 4575 images belonging to 6 classes.


# Modeling
I will build a basic U-net model, train it on the cityscape public dataset, and use that model & weights on the Denver dataset I have built. 

Model structure reference [here](http://cs230.stanford.edu/files_winter_2018/projects/6937642.pdf)

Next, I will use the frozen weights from the first have of the model, 

In [0]:
img_shape = (*target_size, 3)

In [0]:
#use function for the blocks of the network
def encoder_builder(input_, filters,
                         activ='relu', kernel=(3,3), 
                         drop=.5, pad='same', kern_init='he_uniform'):
  kwargs = {'filters': filters, 'activation': activ, 'kernel_size': kernel, 
       'padding': pad, 'kernel_initializer': kern_init}
  x = Conv2D(**kwargs)(input_)
  x = BatchNormalization()(x)
  x = Dropout(drop)(x)
  x = Conv2D(**kwargs)(x)
  encoder = Dropout(drop)(x)
  pooled = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(encoder)
  return encoder, pooled

def decoder_builder(input_, skip, filters, 
                    activ='relu', kernel=(2,2),
                    drop=.5, pad='same', kern_init='he_uniform',
                    ):
  kwargs = {'filters': filters, 'activation': activ, 'kernel_size': kernel, 
       'padding': pad, 'kernel_initializer': kern_init} 
  x = Conv2DTranspose(**kwargs, strides=(2,2))(input_)
  x = concatenate([x, skip], axis=-1)  # note axis is *-*1
  x = BatchNormalization()(x)
  x = Dropout(drop)(x)
  x = Conv2D(**kwargs)(x)
  x = BatchNormalization()(x)
  x = Dropout(drop)(x)
  x = Conv2D(**kwargs)(x)
  return x

In [0]:
input_layer = Input(shape=img_shape)
encoder1, pooled1 = encoder_builder(input_layer, filters=32)  # return (128x128x32)
encoder2, pooled2 = encoder_builder(pooled1, filters=64)  # return (64x64x364)
encoder3, pooled3 = encoder_builder(pooled2, filters=128)  # return (32x32x128)
encoder4, pooled4 = encoder_builder(pooled3, filters=256)  # return (16x16x256)
encoder5, pooled5 = encoder_builder(pooled4, filters=512)  # return (8x8x512)
middle, middle_pool = encoder_builder(pooled5, filters=1024)  # return (4x4x1024)
decoder512 = decoder_builder(middle, skip=encoder5, filters=512)
decoder256 = decoder_builder(decoder512, skip=encoder4, filters=256)
decoder128 = decoder_builder(decoder256, skip=encoder3, filters=128)
decoder64 = decoder_builder(decoder128, skip=encoder2, filters=64)
decoder32 = decoder_builder(decoder64, skip=encoder1, filters=32)
out_layer = Conv2D(filters=1, kernel_size=(1, 1), 
                   activation='sigmoid')(decoder32)

In [0]:
unet = models.Model(inputs=[input_layer], outputs=[out_layer])

In [0]:
unet.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

unet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 256, 256, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
from keras.utils import plot_model
plot_model(unet, to_file='model.png')


Using TensorFlow backend.


TypeError: ignored

In [0]:
os.getcwd()

### Training

In [0]:
n_epochs = 25
learning_rate = 0.000_001

In [0]:
#generator based caluculations
input_shape = train_generator.next()[0].shape[1:]
total_train_samples = train_generator.n
total_vdate_samples = vdate_generator.n
input_shape

(256, 256, 3)

In [0]:
unet_history = unet.fit_generator(
    train_generator, 
    steps_per_epoch=int(total_train_samples/batch_size),
    epochs=n_epochs,
    verbose=1,
    validation_data=vdate_generator,
    validation_steps=int(total_vdate_samples/batch_size))

Epoch 1/25


ResourceExhaustedError: ignored